## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [1]:
import torch
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from collections import Counter
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset, get_dataset_split_names
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

In [4]:
#Caricamento del dataset. Per ogni split possibile stampo 5 esempi e il numero di elementi che costituiscono lo split.
available_splits = get_dataset_split_names("rotten_tomatoes")
print(available_splits)

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

for split_name, split_data in dataset.items():
    print(f"\n{split_name.upper()}:")
    print(f"Numero di esempi: {len(split_data)}")
    print(f"Features: {split_data.features}")
    print(f"Primi 5 esempi:")
    for i in range(min(5, len(split_data))):
          example = split_data[i]
          print(f"Esempio {i+1}: Label={example['label']}, Text='{example['text'][:80]}...'")

['train', 'validation', 'test']

TRAIN:
Numero di esempi: 8530
Features: {'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}
Primi 5 esempi:
Esempio 1: Label=1, Text='the rock is destined to be the 21st century's new " conan " and that he's going ...'
Esempio 2: Label=1, Text='the gorgeously elaborate continuation of " the lord of the rings " trilogy is so...'
Esempio 3: Label=1, Text='effective but too-tepid biopic...'
Esempio 4: Label=1, Text='if you sometimes like to go to the movies to have fun , wasabi is a good place t...'
Esempio 5: Label=1, Text='emerges as something rare , an issue movie that's so honest and keenly observed ...'

VALIDATION:
Numero di esempi: 1066
Features: {'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}
Primi 5 esempi:
Esempio 1: Label=1, Text='compassionately explores the seemingly irreconcilable situation between conserva...'
Esempio 2: Label=1, Text='the soundtrack alone is worth the price of admission ....'
Esempio 3:

#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [5]:
#Caricamento del modello e del tokenizer.
model = AutoModel.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

#Passo 10 esempi al tokenizzatore i risultati ottenuti.
for i in range(10):
    text = dataset['train'][i]['text']
    tok_inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**tok_inputs)
    print(outputs)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BaseModelOutput(last_hidden_state=tensor([[[-0.0332, -0.0168,  0.0194,  ...,  0.0476,  0.5834,  0.3036],
         [-0.0235, -0.0555, -0.3638,  ...,  0.1877,  0.5781, -0.1577],
         [-0.0516, -0.1014, -0.1511,  ...,  0.1503,  0.2649, -0.1575],
         ...,
         [-0.2214,  0.0666, -0.1378,  ...,  0.0319,  0.0833, -0.2145],
         [ 0.6647,  0.2524,  0.0299,  ...,  0.0841, -0.4030, -0.4060],
         [ 0.3342,  0.5060,  0.4131,  ...,  0.1109, -0.2385, -0.2486]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)
BaseModelOutput(last_hidden_state=tensor([[[-0.2062, -0.0490, -0.4036,  ..., -0.1186,  0.6141,  0.3919],
         [-0.4361, -0.1647, -0.3533,  ...,  0.1086,  0.9478, -0.0272],
         [-0.1164,  0.1690,  0.2698,  ..., -0.1971,  0.4372,  0.2527],
         ...,
         [-0.2341,  0.4810, -0.2634,  ..., -0.3397,  0.2567,  0.1274],
         [ 0.7139,  0.0574, -0.3260,  ...,  0.2041, -0.3800, -0.3343],
         [ 0.5649,  0.2806, -0.0295,  ..

#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [6]:
#Utilizzo il modello come feature extractor per stabilire una baseline. Innanzitutto ripartisco i dati in base agli split individuati precedentemente.
train_data, train_labels = dataset['train']['text'], dataset['train']['label']
val_data, val_labels = dataset['validation']['text'], dataset['validation']['label']
test_data, test_labels = dataset['test']['text'], dataset['test']['label']

feature_extractor = pipeline(
    'feature-extraction',
    model=model,
    tokenizer=tokenizer,
    return_all_layers=False #ritorna solo l'ultimo layer, ovvero quello di interesse per l'estrazione del token [CLS]
)

#Creo una funzione per l'estrazione delle features utilizzando la pipeline in batch ed estraggo solamente il token [CLS].
def extract_cls_features(data, batch_size=16):
  features = []
  for i in range(0, len(data), batch_size):
      batch_texts = data[i:i + batch_size]
      batch_outputs = feature_extractor(batch_texts)

      for output in batch_outputs:
          output = np.array(output)
          cls_embedding = output[0, 0, :]
          features.append(cls_embedding)
  return np.vstack(features)


x_train = extract_cls_features(train_data)
x_val = extract_cls_features(val_data)
x_test = extract_cls_features(test_data)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

svm = SVC()
svm.fit(x_train, y_train) #addestro il modello base con le features estratte

val_pred = svm.predict(x_val)
test_pred = svm.predict(x_test)

val_acc = accuracy_score(y_val, val_pred) #valuto le predizioni sul validation set
test_acc = accuracy_score(y_test, test_pred) #valuto le predizioni sul test set

print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

print(classification_report(
    y_val,
    val_pred,
    target_names=['Negative', 'Positive'],
    digits=3
))

print(classification_report(
    y_test,
    test_pred,
    target_names=['Negative', 'Positive'],
    digits=3
))

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Validation accuracy: 0.8143
Test accuracy: 0.7946
              precision    recall  f1-score   support

    Negative      0.785     0.865     0.823       533
    Positive      0.850     0.764     0.804       533

    accuracy                          0.814      1066
   macro avg      0.818     0.814     0.814      1066
weighted avg      0.818     0.814     0.814      1066

              precision    recall  f1-score   support

    Negative      0.772     0.837     0.803       533
    Positive      0.822     0.752     0.786       533

    accuracy                          0.795      1066
   macro avg      0.797     0.795     0.794      1066
weighted avg      0.797     0.795     0.794      1066



-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [7]:
tokenized_set = dataset.map(lambda examples: tokenizer(examples["text"], truncation=True, padding=False), batched=True)
tokenized_set = tokenized_set.rename_column("label", "labels")
train_split_tokenized = tokenized_set['train']
val_split_tokenized = tokenized_set['validation']
test_split_tokenized = tokenized_set['test']
print(train_split_tokenized[0].keys())
print(val_split_tokenized[0].keys())
print(test_split_tokenized[0].keys())

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

dict_keys(['text', 'labels', 'input_ids', 'attention_mask'])
dict_keys(['text', 'labels', 'input_ids', 'attention_mask'])
dict_keys(['text', 'labels', 'input_ids', 'attention_mask'])


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [19]:
#Creazione del modello per il finetuning.
classification_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,  #classificazione binaria
    id2label={0: "Negative", 1: "Positive"}, #mapping
    label2id={"Negative": 0, "Positive": 1} #mapping
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [21]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)

#Definizione della funzione di valutazione che misura accuracy, precision, recall e F1 score.
def evaluation_function(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    preds = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

#Si riducono il learning rate e le epoche di training perchè il modello è già preaddestrato, quindi si vuole evitare overfitting.
training_args = TrainingArguments(
    num_train_epochs = 3,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    optim = "adamw_torch",
    learning_rate = 2e-5,
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model = classification_model,
    args = training_args,
    train_dataset = train_split_tokenized,
    eval_dataset = val_split_tokenized,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = evaluation_function
)

trainer.train()
trainer.evaluate()
test_results = trainer.evaluate(eval_dataset=test_split_tokenized)
print("Test results:")
print(test_results)

/tmp/ipython-input-2704273867.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
1,No log,0.445297,0.847092,0.850050,0.847092,0.846768
2,0.201000,0.437230,0.849906,0.851336,0.849906,0.849753
3,0.201000,0.505701,0.852720,0.852871,0.852720,0.852705


Test results:
{'eval_loss': 0.5666215419769287, 'eval_accuracy': 0.8339587242026266, 'eval_precision': 0.8339693044212898, 'eval_recall': 0.8339587242026266, 'eval_f1_score': 0.8339574091364698, 'eval_runtime': 0.7455, 'eval_samples_per_second': 1429.92, 'eval_steps_per_second': 45.607, 'epoch': 3.0}


Si osserva che l'accuracy della baseline valutata sul test set è del 79% mentre l'accuracy del modello dopo il finetuning è dell'83%. Questo mostra un miglioramento significativo.

-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [23]:
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
)

lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, target_modules=["q_lin", "v_lin"], r=16, lora_alpha=32)
lora_classification_model = get_peft_model(classification_model, lora_config)

trainer = Trainer(
    model = lora_classification_model,
    args = training_args,
    train_dataset = train_split_tokenized,
    eval_dataset = val_split_tokenized,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = evaluation_function
)

train_result = trainer.train()
eval_results = trainer.evaluate()
test_results = trainer.evaluate(eval_dataset=test_split_tokenized)
print("Test results:")
print(test_results)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipython-input-3559789563.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
1,No log,0.532713,0.851782,0.851862,0.851782,0.851774
2,0.045500,0.547756,0.848968,0.848969,0.848968,0.848968
3,0.045500,0.552086,0.848030,0.848050,0.848030,0.848028


Test results:
{'eval_loss': 0.6266307234764099, 'eval_accuracy': 0.8330206378986866, 'eval_precision': 0.8330956781973418, 'eval_recall': 0.8330206378986866, 'eval_f1_score': 0.8330112330281301, 'eval_runtime': 0.7941, 'eval_samples_per_second': 1342.438, 'eval_steps_per_second': 42.817, 'epoch': 3.0}


Anche dopo aver effettuato il finetuning con Lora si osserva che l'accuracy è dell'83%. Di fatto sono stati ottenuti gli stessi risultati del finetuning completo su Distilbert utilizzando un minor numero di parametri.

#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [ ]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

In [ ]:
# Your code here.